<a href="https://colab.research.google.com/github/kevorkian-mano/Motor_Vehicle_Collisions_Project/blob/main/wprk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dash==2.15.0 dash-bootstrap-components pyngrok plotly pandas


INFO: pip is looking at multiple versions of dash-bootstrap-components to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 823.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.2
    Uninstalling Flask-3.1.2:
      Successfully uninstalled Flask-3.1.2


In [ ]:
from pyngrok import ngrok

# paste your token:
ngrok.set_auth_token("35UVQIkvrzthd18TJK5RWg9cpop_63kdb9Dev1GJJixRfUpk1")


In [ ]:
# app_nyc_crash_dashboard.py
import ast
import numpy as np
import pandas as pd
import dash
from dash import dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
from pyngrok import ngrok # optional - for local tunneling
# ------------------------------------------------------------------
# Load dataset
# ------------------------------------------------------------------
df = pd.read_csv("merged_cleaned_dataset.csv", dtype=str) # load as strings to be safe
# Normalize and cast useful columns
# Keep original columns but create convenient working columns
# Some columns have spaces in names; use exact names from your message.
# Convert crash datetime to datetime (coerce errors)
df["CRASH_DATETIME"] = pd.to_datetime(df["CRASH_DATETIME"], errors="coerce")
# YEAR for slider and groupings
df["YEAR"] = df["CRASH_DATETIME"].dt.year
# Cast numeric injury/killed counts to numeric (safe)
num_cols = [
     "NUMBER OF PERSONS INJURED", "NUMBER OF PERSONS KILLED",
     "NUMBER OF PEDESTRIANS INJURED", "NUMBER OF PEDESTRIANS KILLED",
     "NUMBER OF CYCLIST INJURED", "NUMBER OF CYCLIST KILLED",
     "NUMBER OF MOTORIST INJURED", "NUMBER OF MOTORIST KILLED"
]
for c in num_cols:
     if c in df.columns:
          df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype(int)
     else:
          df[c] = 0
# Helpful aggregated numeric columns
df["TOTAL_INJURED"] = df[["NUMBER OF PERSONS INJURED",
                         "NUMBER OF PEDESTRIANS INJURED",
                         "NUMBER OF CYCLIST INJURED",
                         "NUMBER OF MOTORIST INJURED"]].sum(axis=1)
df["TOTAL_KILLED"] = df[["NUMBER OF PERSONS KILLED",
                         "NUMBER OF PEDESTRIANS KILLED",
                         "NUMBER OF CYCLIST KILLED",
                         "NUMBER OF MOTORIST KILLED"]].sum(axis=1)
# Safe borough column
if "BOROUGH" not in df.columns:
     df["BOROUGH"] = "Unknown"
df["BOROUGH"] = df["BOROUGH"].fillna("Unknown")
# FULL_ADDRESS fallback
if "FULL ADDRESS" not in df.columns:
     df["FULL ADDRESS"] = df.get("ON STREET NAME", "").fillna("") + ", " + df.get("BOROUGH", "")
# Latitude / Longitude as numeric
for coord in ("LATITUDE", "LONGITUDE"):
     if coord in df.columns:
          df[coord] = pd.to_numeric(df[coord], errors="coerce")
     else:
          df[coord] = np.nan
# Parse ALL_VEHICLE_TYPES (which may be a string representation of a list) and create a flattened column
def parse_vehicle_list(v):
     if pd.isna(v):
          return []
     # If it's already a Python list object (rare in CSV), handle
     if isinstance(v, list):
          return [str(x).strip() for x in v if str(x).strip()]
     s = str(v).strip()
     # Try literal_eval if it's like "['SUV/Station Wagon', 'Sedan']"
     try:
          parsed = ast.literal_eval(s)
          if isinstance(parsed, (list, tuple)):
               return [str(x).strip() for x in parsed if str(x).strip()]
     except Exception:
          # fallback: comma-separated
          parts = [p.strip() for p in s.split(",") if p.strip()]
          return parts
     return []
df["VEHICLE_TYPES_LIST"] = df.get("ALL_VEHICLE_TYPES", "").apply(parse_vehicle_list)
# Expand vehicle types per row into a flat list column for easier counting
all_vehicle_types_flat = [vt for sub in df["VEHICLE_TYPES_LIST"] for vt in sub]
vehicle_type_counts = pd.Series(all_vehicle_types_flat).value_counts()
# Top 10 vehicle types for charts / heatmap combos
TOP_VEHICLE_TYPES = vehicle_type_counts.head(10).index.tolist()
# Parse contributing factors (all contributing factors column may be a list or string)
def parse_factor_list(v):
     if pd.isna(v):
          return []
     if isinstance(v, list):
          return [str(x).strip() for x in v if str(x).strip()]
     s = str(v).strip()
     try:
          parsed = ast.literal_eval(s)
          if isinstance(parsed, (list, tuple)):
               return [str(x).strip() for x in parsed if str(x).strip()]
     except Exception:
          parts = [p.strip() for p in s.split(",") if p.strip()]
          return parts
     return []
# Try to handle both ALL_CONTRIBUTING_FACTORS and ALL_CONTRIBUTING_FACTORS_STR
if "ALL_CONTRIBUTING_FACTORS" in df.columns:
     df["FACTORS_LIST"] = df["ALL_CONTRIBUTING_FACTORS"].apply(parse_factor_list)
elif "ALL_CONTRIBUTING_FACTORS_STR" in df.columns:
     df["FACTORS_LIST"] = df["ALL_CONTRIBUTING_FACTORS_STR"].apply(parse_factor_list)
else:
     # fallback to specific columns if provided
     parts = []
     for i in range(1, 4):
          c = f"CONTRIBUTING FACTOR VEHICLE {i}"
          if c in df.columns:
               parts.append(df[c].fillna("").astype(str))
     if parts:
          df["FACTORS_LIST"] = (pd.Series([";".join(x) for x in zip(*parts)]) if parts else pd.Series([[]]*len(df))).apply(
               lambda s: parse_factor_list(s))
     else:
          df["FACTORS_LIST"] = [[] for _ in range(len(df))]
all_factors_flat = [f for sub in df["FACTORS_LIST"] for f in sub]
factor_counts = pd.Series(all_factors_flat).value_counts()
TOP_FACTORS = factor_counts.head(10).index.tolist()
# PERSON_TYPE (type of persons involved)
if "PERSON_TYPE" not in df.columns and "PERSON_TYPE" in df.columns:
     pass
# ensure PERSON_TYPE column exists
if "PERSON_TYPE" not in df.columns:
     if "PERSON_TYPE" in df.columns:
          df["PERSON_TYPE"] = df["PERSON_TYPE"]
     else:
          df["PERSON_TYPE"] = df.get("PERSON_TYPE", "Unknown").fillna("Unknown")
# POSITION_IN_VEHICLE_CLEAN is provided in dataset per your list, ensure it's present
if "POSITION_IN_VEHICLE_CLEAN" not in df.columns:
     df["POSITION_IN_VEHICLE_CLEAN"] = df.get("POSITION_IN_VEHICLE_CLEAN", "").fillna("Unknown")
# Ensure other person-related columns exist (for new plots)
for col in ["PERSON_AGE", "PERSON_SEX", "BODILY_INJURY", "SAFETY_EQUIPMENT", "EMOTIONAL_STATUS", "UNIQUE_ID", "EJECTION", "ZIP CODE", "PERSON_INJURY"]:
    if col not in df.columns:
        # Create a placeholder column if not found (assuming person-level data is in the merged set)
        if col == "UNIQUE_ID":
            df[col] = df.index + 1
        elif col == "PERSON_AGE":
            df[col] = pd.to_numeric(df.get(col, np.nan), errors='coerce').fillna(0).astype(int) # Coerce age to int, fill missing/bad with 0
        elif col in ["EJECTION", "ZIP CODE", "PERSON_INJURY"]:
             df[col] = df.get(col, "Unknown").fillna("Unknown")
        else:
            df[col] = df.get(col, "Unknown").fillna("Unknown")
# Small helper to add jitter to lat/lon to separate overlapping points
def jitter_coords(series, scale=0.0006):
     # scale tuned for city-level jitter
     return series + np.random.normal(loc=0, scale=scale, size=series.shape)
# ------------------------------------------------------------------
# Create Dash app layout
# ------------------------------------------------------------------
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])
server = app.server
# Year slider marks
min_year = int(df["YEAR"].min()) if not df["YEAR"].isna().all() else 2010
max_year = int(df["YEAR"].max()) if not df["YEAR"].isna().all() else pd.Timestamp.now().year
year_marks = {y: str(y) for y in range(min_year, max_year + 1)}
app.layout = dbc.Container([
     html.H2("NYC Crash Analysis Dashboard", className="mt-3 mb-3"),
     dbc.Row([
          dbc.Col([
               html.Label("Year range"),
               dcc.RangeSlider(
                    id="year_slider",
                    min=min_year,
                    max=max_year,
                    value=[min_year, max_year],
                    marks=year_marks,
                    tooltip={"placement": "bottom", "always_visible": False},
                    step=1,
                    allowCross=False
               ),
          ], width=8),
          dbc.Col([
               html.Label("Borough"),
               dcc.Dropdown(
                    id="borough_filter",
                    options=[{"label": b, "value": b} for b in sorted(df["BOROUGH"].dropna().unique())],
                    multi=True,
                    placeholder="Select borough(s)"
               )
          ], width=4),
     ], className="mb-3"),
     dbc.Row([
     dbc.Col([
          html.Label("Vehicle Type"),
          dcc.Dropdown(
               id="vehicle_filter",
               options=[{"label": v, "value": v}
                        for v in sorted({vt for sub in df["VEHICLE_TYPES_LIST"] for vt in sub})],
               multi=True,
               placeholder="Select vehicle type(s)"
          )
     ], width=6),
     dbc.Col([
          html.Label("Contributing Factor"),
          dcc.Dropdown(
               id="factor_filter",
               options=[{"label": f, "value": f}
                        for f in sorted({f for sub in df["FACTORS_LIST"] for f in sub})],
               multi=True,
               placeholder="Select contributing factor(s)"
          )
     ], width=6),
], className="mb-3"),
     dbc.Row([
          dbc.Col([
               html.Label("Injury Type / Person Injury"),
               dcc.Dropdown(
                    id="injury_filter",
                    options=[{"label": i, "value": i} for i in sorted(df["PERSON_INJURY"].dropna().unique())],
                    multi=True,
                    placeholder="Select injury type(s)"
               )
          ], width=6),
          dbc.Col([
               html.Label("Search (e.g., 'Brooklyn 2019 pedestrian')"),
               dbc.Input(id="search_input", placeholder="Search text", type="text")
          ], width=6),
     ], className="mb-3"),
     dbc.Row([
          dbc.Col(dbc.Button("Generate Report / Update", id="generate_btn", color="primary"), width=3),
          dbc.Col(html.Div(id="summary_text", style={"paddingTop": "8px"}), width=9)
     ], className="mb-3"),
     # Charts organized in cards to separate them visually
     dbc.Row([
          dbc.Col(dbc.Card(dbc.CardBody([html.H5("Crashes by Borough"),
                                         dcc.Graph(id="crashes_by_borough")])), md=6),
          dbc.Col(dbc.Card(dbc.CardBody([html.H5("Injuries by Borough"),
                                         dcc.Graph(id="injuries_by_borough")])), md=6),
     ], className="mb-3"),
     dbc.Row([
          dbc.Col(dbc.Card(dbc.CardBody([html.H5("Crashes by Contributing Factor (Top)"),
                                         dcc.Graph(id="crashes_by_factor")])), md=6),
          dbc.Col(dbc.Card(dbc.CardBody([html.H5("Injuries by Position in Vehicle"),
                                         dcc.Graph(id="injuries_by_position")])), md=6),
     ], className="mb-3"),
     # --- NEW PLOT 1 (Stacked Borough Stats) ---
     dbc.Row([
         dbc.Col(dbc.Card(dbc.CardBody([html.H5("Injuries & Fatalities by Borough (Stacked)"),
                                       dcc.Graph(id="injuries_by_borough_stacked")])), md=12),
     ], className="mb-3"),
     # --- NEW PLOT 2 (Injuries/Killed Over Time) ---
     dbc.Row([
         dbc.Col(dbc.Card(dbc.CardBody([html.H5("Total Injuries & Fatalities Over Time"),
                                       dcc.Graph(id="injuries_killed_over_time")])), md=12),
     ], className="mb-3"),
     # --- NEW PLOT 3 (Age Distribution with Stats) ---
     dbc.Row([
         dbc.Col(dbc.Card(dbc.CardBody([html.H5("Age Distribution with Statistics"),
                                       dcc.Graph(id="age_distribution_hist")])), md=12),
     ], className="mb-3"),
     # --- RE-ADDED: Gender Distribution (as requested) ---
     dbc.Row([
          dbc.Col(dbc.Card(dbc.CardBody([html.H5("Gender Distribution in Crashes"),
                                         dcc.Graph(id="gender_distribution")])), md=6),
          dbc.Col(dbc.Card(dbc.CardBody([html.H5("Injury Severity Distribution"),
                                         dcc.Graph(id="injury_severity")])), md=6),
     ], className="mb-3"),
     # --- NEW PLOT 4 (ZIP Code Choropleth) ---
     dbc.Row([
         dbc.Col(dbc.Card(dbc.CardBody([html.H5("Crashes by ZIP Code"),
                                       dcc.Graph(id="zip_code_choropleth")])), md=12),
     ], className="mb-3"),
     # --- ORIGINAL PLOT 4 & 6 ---
     dbc.Row([
          dbc.Col(dbc.Card(dbc.CardBody([html.H5("Safety Equipment Usage"),
                                         dcc.Graph(id="safety_equipment")])), md=6),
          dbc.Col(dbc.Card(dbc.CardBody([html.H5("Emotional State Distribution in Crashes"),
                                         dcc.Graph(id="emotional_state")])), md=6),
     ], className="mb-3"),
     dbc.Row([
          dbc.Col(dbc.Card(dbc.CardBody([html.H5("Crashes per Year (by Borough)"),
                                         dcc.Graph(id="crashes_by_year")])), md=12),
     ], className="mb-3"),
     dbc.Row([
          dbc.Col(dbc.Card(dbc.CardBody([html.H5("Vehicle Type Distribution (Pie)"),
                                         dcc.Graph(id="vehicle_pie")])), md=6),
          dbc.Col(dbc.Card(dbc.CardBody([html.H5("Person Types Involved (Pie)"),
                                         dcc.Graph(id="person_type_pie")])), md=6),
     ], className="mb-3"),
     # --- NEW PLOT 5 (Injuries by Person Type Over Time) ---
     dbc.Row([
         dbc.Col(dbc.Card(dbc.CardBody([html.H5("Injuries by Person Type Over Time"),
                                       dcc.Graph(id="injuries_by_person_type")])), md=12),
     ], className="mb-3"),
     # --- ORIGINAL PLOT 5 (Expanded Heatmap) ---
     dbc.Row([
          dbc.Col(dbc.Card(dbc.CardBody([html.H5("Vehicle Type vs Contributing Factors (Expanded)"),
                                         dcc.Graph(id="expanded_veh_factor")])), md=12),
     ], className="mb-3"),
     # --- NEW PLOT 6 (Ejection Status Distribution) ---
     dbc.Row([
          dbc.Col(dbc.Card(dbc.CardBody([html.H5("Ejection Status Distribution"),
                                        dcc.Graph(id="ejection_status")])), md=6),
     ], className="mb-3"),
     dbc.Row([
          dbc.Col(dbc.Card(dbc.CardBody([html.H5("Crash Density Map (heat)"),
                                         dcc.Graph(id="density_map")])), md=6),
          dbc.Col(dbc.Card(dbc.CardBody([html.H5("Crash Locations Map (points)"),
                                         dcc.Graph(id="map_chart")])), md=6),
     ], className="mb-4"),
     # Hidden divs etc if needed
], fluid=True)
# -------------------------
# Helper: parse search query
# -------------------------
def parse_search_query(q):
     q = (q or "").lower()
     found = {}
     # borough detection
     for b in df["BOROUGH"].dropna().unique():
          if b and b.lower() in q:
               found["borough"] = [b]
     # year detection
     for y in df["YEAR"].dropna().unique():
          try:
               if str(int(y)) in q:
                    found["year"] = [int(y)]
          except Exception:
               pass
     if "pedestrian" in q:
          # a heuristic: treat pedestrian as injured/killed persons (you can adjust)
          found["injury"] = ["Injured", "Killed"]
     return found
# -------------------------
# Callback: generate figures
# -------------------------
@app.callback(
     [
          Output("crashes_by_borough", "figure"),
          Output("injuries_by_borough", "figure"),
          Output("crashes_by_factor", "figure"),
          Output("injuries_by_position", "figure"),
          Output("crashes_by_year", "figure"),
          Output("vehicle_pie", "figure"),
          Output("person_type_pie", "figure"),
          # Removed: Output("veh_factor_heatmap", "figure"),
          Output("density_map", "figure"),
          Output("map_chart", "figure"),
          # Removed: Output("age_distribution", "figure"),
          Output("gender_distribution", "figure"), # RE-ADDED
          Output("injury_severity", "figure"),
          Output("safety_equipment", "figure"),
          Output("expanded_veh_factor", "figure"),
          Output("emotional_state", "figure"),
          # NEW PLOTS ADDED TO OUTPUT
          Output("injuries_by_borough_stacked", "figure"),
          Output("injuries_killed_over_time", "figure"),
          Output("age_distribution_hist", "figure"),
          Output("zip_code_choropleth", "figure"),
          Output("injuries_by_person_type", "figure"),
          Output("ejection_status", "figure"),
          # END OF NEW PLOTS
          Output("summary_text", "children"),
     ],
     Input("generate_btn", "n_clicks"),
     [
          State("year_slider", "value"),
          State("borough_filter", "value"),
          State("vehicle_filter", "value"),
          State("factor_filter", "value"),
          State("injury_filter", "value"),
          State("search_input", "value"),
     ]
)
def update_dashboard(n_clicks, year_range, boroughs, vehicles, factors, injuries, search_text):
     dff = df.copy()
     # --- Apply search query (intersect with selected filters) ---
     if search_text:
          parsed = parse_search_query(search_text)
          # Borough filter
          if "borough" in parsed:
               boroughs = list(set(boroughs or dff["BOROUGH"].dropna().unique()) & set(parsed["borough"]))
          # Year filter
          if "year" in parsed:
               yr = parsed["year"]
               if yr:
                    year_range = [
                         max(year_range[0], int(yr[0])),
                         min(year_range[1], int(yr[0]))
                    ]
          # Injuries filter
          if "injury" in parsed:
               injuries = list(set(injuries or dff["PERSON_INJURY"].dropna().unique()) & set(parsed["injury"]))
     # --- Apply year range ---
     if year_range and len(year_range) == 2:
          y0, y1 = int(year_range[0]), int(year_range[1])
          dff = dff[(dff["YEAR"] >= y0) & (dff["YEAR"] <= y1)]
     # --- Apply borough filter ---
     if boroughs:
          dff = dff[dff["BOROUGH"].isin(boroughs)]
     # --- Apply injury filter ---
     if injuries:
          dff = dff[dff["PERSON_INJURY"].fillna("").astype(str).isin([str(i) for i in injuries])]
     # --- Apply vehicle filter ---
     if vehicles:
          mask = dff["VEHICLE_TYPES_LIST"].apply(lambda lst: any(v in (lst if isinstance(lst, list) else []) for v in vehicles))
          dff = dff[mask]
     # --- Apply contributing factor filter ---
     if factors:
          mask = dff["FACTORS_LIST"].apply(lambda lst: any(f in (lst if isinstance(lst, list) else []) for f in factors))
          dff = dff[mask]
     # ---------- Figures ----------
     # 1) Crashes by Borough
     crashes_by_borough = dff.groupby("BOROUGH").size().reset_index(name="Crashes").sort_values("Crashes", ascending=False)
     fig_borough = px.bar(crashes_by_borough, x="BOROUGH", y="Crashes",
                          title="Crashes by Borough", labels={"Crashes": "Number of Crashes", "BOROUGH": "Borough"},
                          text="Crashes")
     fig_borough.update_traces(textposition="outside")
     fig_borough.update_layout(margin=dict(t=40, b=20))
     # 2) Injuries by Borough
     injuries_by_borough = dff.groupby("BOROUGH")["TOTAL_INJURED"].sum().reset_index().sort_values("TOTAL_INJURED", ascending=False)
     fig_inj_borough = px.bar(injuries_by_borough, x="BOROUGH", y="TOTAL_INJURED",
                              title="Total Injuries by Borough",
                              labels={"TOTAL_INJURED": "Total Injured", "BOROUGH": "Borough"},
                              text="TOTAL_INJURED")
     fig_inj_borough.update_traces(textposition="outside")
     fig_inj_borough.update_layout(margin=dict(t=40, b=20))
     # 3) Crashes by Contributing Factor (top 15)
     factor_rows = []
     for _, row in dff.iterrows():
          for f in row["FACTORS_LIST"]:
               factor_rows.append((f, row["UNIQUE_ID"] if "UNIQUE_ID" in row else 1))
     factor_df = pd.DataFrame(factor_rows, columns=["Factor", "UID"]) if factor_rows else pd.DataFrame(columns=["Factor", "UID"])
     factor_counts_df = factor_df["Factor"].value_counts().head(15).reset_index()
     factor_counts_df.columns = ["Factor", "Count"]
     fig_factor = px.bar(factor_counts_df, x="Count", y="Factor", orientation="h",
                         title="Top Contributing Factors (Top 15)",
                         labels={"Count": "Number of Crashes", "Factor": "Contributing Factor"})
     fig_factor.update_layout(margin=dict(t=40, b=20), yaxis={'categoryorder':'total ascending'})
     # 4) Injuries by Position in Vehicle
     pos_df = dff.groupby("POSITION_IN_VEHICLE_CLEAN")["TOTAL_INJURED"].sum().reset_index().sort_values("TOTAL_INJURED", ascending=False)
     fig_pos = px.bar(pos_df, x="POSITION_IN_VEHICLE_CLEAN", y="TOTAL_INJURED",
                      title="Injuries by Position in Vehicle",
                      labels={"TOTAL_INJURED": "Total Injured", "POSITION_IN_VEHICLE_CLEAN": "Position"})
     fig_pos.update_layout(xaxis_tickangle=-45, margin=dict(t=40, b=80))
     # 5) Crashes per Year (line) by Borough
     year_group = dff.groupby(["YEAR", "BOROUGH"]).size().reset_index(name="Crashes")
     if not year_group.empty:
          fig_year = px.line(year_group, x="YEAR", y="Crashes", color="BOROUGH",
                             title="Crashes per Year (by Borough)", markers=True)
     else:
          fig_year = go.Figure()
          fig_year.update_layout(title="Crashes per Year (no data for selection)")
     # 6) Vehicle Type Distribution (pie)
     veh_rows = [vt for sub in dff["VEHICLE_TYPES_LIST"] for vt in sub]
     veh_counts = pd.Series(veh_rows).value_counts()
     if veh_counts.empty:
          fig_veh_pie = go.Figure()
          fig_veh_pie.update_layout(title="Vehicle Type Distribution (no data)")
     else:
          labels = veh_counts.index.tolist()
          values = veh_counts.values.tolist()
          fig_veh_pie = px.pie(names=labels, values=values, title="Vehicle Type Distribution - for filtered vechile types it includes other vechiles that where also involved in the crash")
     # 7) Person type pie
     if "PERSON_TYPE" in dff.columns:
          person_counts = dff["PERSON_TYPE"].fillna("Unknown").value_counts()
          fig_person = px.pie(names=person_counts.index, values=person_counts.values, title="Person Types Involved")
     else:
          fig_person = go.Figure()
          fig_person.update_layout(title="No PERSON_TYPE column found")
     # 8) Vehicle Type vs Contributing Factor heatmap (REMOVED - now handled by fig_expanded)
     # 9) Density map
     df_map = dff.dropna(subset=["LATITUDE", "LONGITUDE"]).copy()
     if not df_map.empty:
          fig_density = px.density_mapbox(df_map, lat="LATITUDE", lon="LONGITUDE", radius=8,
                                          center={"lat": 40.7128, "lon": -74.0060},
                                          zoom=9, hover_data=["FULL ADDRESS", "TOTAL_INJURED", "TOTAL_KILLED", "CRASH_DATETIME"],
                                          mapbox_style="open-street-map",
                                          title="Crash Density (Heat)")
     else:
          fig_density = go.Figure()
          fig_density.update_layout(title="No location data to show density")
     # 10) Crash locations points map
     if not df_map.empty:
          df_map["_LAT_JIT"] = jitter_coords(df_map["LATITUDE"].fillna(0).astype(float), scale=0.0005)
          df_map["_LON_JIT"] = jitter_coords(df_map["LONGITUDE"].fillna(0).astype(float), scale=0.0005)
          fig_map = px.scatter_mapbox(df_map, lat="_LAT_JIT", lon="_LON_JIT", color="BOROUGH",
                                       hover_name="FULL ADDRESS",
                                       hover_data={"FULL ADDRESS": True,
                                                   "TOTAL_INJURED": True,
                                                   "TOTAL_KILLED": True,
                                                   "CRASH_DATETIME": True,
                                                   "_LAT_JIT": False, "_LON_JIT": False},
                                       zoom=9, height=600,
                                       title="Crash Locations (points)",
                                       mapbox_style="open-street-map")
          fig_map.update_traces(marker=dict(size=6, opacity=0.6))
          fig_map.update_layout(margin=dict(t=40))
     else:
          fig_map = go.Figure()
          fig_map.update_layout(title="No location data to display")

     # 11) Gender Distribution (RE-ADDED)
     gender_dist = dff.groupby("PERSON_SEX")["UNIQUE_ID"].count().reset_index(name="Count")
     fig_gender = px.pie(gender_dist, names="PERSON_SEX", values="Count", title="Gender Distribution in Crashes")
     fig_gender.update_layout(margin=dict(t=40, b=20))

     # 12) Injury Severity Distribution
     injury_dist = dff.groupby("BODILY_INJURY")["UNIQUE_ID"].count().reset_index(name="Count")
     fig_injury = px.bar(injury_dist, x="BODILY_INJURY", y="Count", title="Injury Severity Distribution",
                          labels={"BODILY_INJURY": "Injury Severity", "Count": "Number of Records"})
     fig_injury.update_layout(margin=dict(t=40, b=20))

     # 13) Safety Equipment Usage
     safety_dist = dff.groupby("SAFETY_EQUIPMENT")["UNIQUE_ID"].count().reset_index(name="Count")
     fig_safety = px.pie(safety_dist, names="SAFETY_EQUIPMENT", values="Count", title="Safety Equipment Usage",
                          labels={"SAFETY_EQUIPMENT": "Safety Equipment", "Count": "Number of Records"})
     fig_safety.update_layout(margin=dict(t=40, b=20))

     # 14) Expanded Vehicle Type vs Contributing Factors
     pair_rows_expanded = []
     for _, row in dff.iterrows():
          vlist = row["VEHICLE_TYPES_LIST"]
          flist = row["FACTORS_LIST"]
          for v in vlist:
               for f in flist:
                    pair_rows_expanded.append((v, f))
     if pair_rows_expanded:
          pair_df_expanded = pd.DataFrame(pair_rows_expanded, columns=["Vehicle", "Factor"])
          pivot_expanded = pair_df_expanded.groupby(["Vehicle", "Factor"]).size().reset_index(name="Count")
          pivot_table_expanded = pivot_expanded.pivot(index="Vehicle", columns="Factor", values="Count").fillna(0)
          fig_expanded = go.Figure(data=go.Heatmap(
               z=pivot_table_expanded.values,
               x=pivot_table_expanded.columns.tolist(),
               y=pivot_table_expanded.index.tolist(),
               hovertemplate="Vehicle: %{y}<br>Factor: %{x}<br>Count: %{z}<extra></extra>"
          ))
          fig_expanded.update_layout(title="Vehicle Type vs Contributing Factors (Expanded View)",
                                     xaxis_nticks=12, margin=dict(t=50))
     else:
          fig_expanded = go.Figure()
          fig_expanded.update_layout(title="No data for Vehicle×Factor heatmap for the current selection")

     # 15) Emotional State Distribution
     emotional_dist = dff.groupby("EMOTIONAL_STATUS")["UNIQUE_ID"].count().reset_index(name="Count")
     fig_emotional = px.bar(emotional_dist, x="EMOTIONAL_STATUS", y="Count", title="Emotional State Distribution in Crashes",
                             labels={"EMOTIONAL_STATUS": "Emotional State", "Count": "Number of Records"})
     fig_emotional.update_layout(margin=dict(t=40, b=20))


     # 16) Injuries & Fatalities by Borough (Stacked)
     borough_stats = dff.groupby("BOROUGH").agg({
         "TOTAL_INJURED": "sum",
         "TOTAL_KILLED": "sum"
     }).reset_index()

     fig_stacked = px.bar(borough_stats, x="BOROUGH", y=["TOTAL_INJURED", "TOTAL_KILLED"],
                         title="Injuries & Fatalities by Borough",
                         barmode="group",
                         labels={"value": "Count", "variable": "Type"},
                         color_discrete_map={"TOTAL_INJURED": "#FF9999", "TOTAL_KILLED": "#660000"})

     fig_stacked.update_layout(
         margin=dict(t=40, b=20),
         legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
     )

     # 17) Total Injuries & Fatalities Over Time
     time_series = dff.groupby("YEAR").agg({
         "TOTAL_INJURED": "sum",
         "TOTAL_KILLED": "sum"
     }).reset_index()

     fig_time = px.line(time_series, x="YEAR", y=["TOTAL_INJURED", "TOTAL_KILLED"],
                       title="Total Injuries & Fatalities Over Time",
                       markers=True,
                       hover_name="YEAR")

     fig_time.update_layout(
         margin=dict(t=40, b=20),
         legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
     )

     # 18) Age Distribution with Marginal Box Plot
     dff["PERSON_AGE"] = pd.to_numeric(dff["PERSON_AGE"], errors='coerce')
     fig_age_hist = px.histogram(dff, x="PERSON_AGE", nbins=30,
                           marginal="box",
                           title="Age Distribution with Statistical Summary",
                           hover_data=["PERSON_AGE"])

     fig_age_hist.update_layout(
         margin=dict(t=40, b=20),
         xaxis_title="Age",
         yaxis_title="Count"
     )

     # 19) ZIP Code Choropleth
     zip_stats = dff[dff["ZIP CODE"].str.isdigit()].groupby("ZIP CODE").size().reset_index(name="Crash Count")

     fig_zip = px.choropleth(zip_stats,
                             locations="ZIP CODE",
                             locationmode="USA-states",  # Placeholder for demonstration
                             color="Crash Count",
                             color_continuous_scale="Reds",
                             scope="usa",
                             title="Crashes by ZIP Code (Demonstration - requires GeoJSON for true accuracy)")

     fig_zip.update_layout(
         margin=dict(t=40, b=20),
         geo=dict(
             showland=True,
             landcolor="lightgray",
             showcoastlines=True,
             coastlinecolor="white"
         )
     )

     # 20) Injuries by Person Type Over Time
     person_type_time = dff.groupby(["YEAR", "PERSON_TYPE"]).agg({
         "TOTAL_INJURED": "sum"
     }).reset_index()

     fig_person_time = px.bar(person_type_time, x="YEAR", y="TOTAL_INJURED", color="PERSON_TYPE",
                             barmode="stack",
                             title="Injuries by Person Type Over Time")

     fig_person_time.update_layout(
         margin=dict(t=40, b=20),
         legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
     )

     # 21) Ejection Status Distribution
     ejection_dist = dff.groupby("EJECTION")["UNIQUE_ID"].count().reset_index(name="Count")
     ejection_dist = ejection_dist.sort_values("Count", ascending=False)

     fig_ejection = px.bar(ejection_dist, x="EJECTION", y="Count",
                          title="Ejection Status Distribution",
                          color_discrete_sequence=["#636EFA"])

     fig_ejection.update_layout(
         margin=dict(t=40, b=20),
         xaxis_title="Ejection Status",
         yaxis_title="Count"
     )

     # --- Summary text generation ---
     total_crashes = len(dff)
     total_injuries = dff["TOTAL_INJURED"].sum()
     total_killed = dff["TOTAL_KILLED"].sum()
     summary = f"**Data Summary:** {total_crashes:,} crashes, {total_injuries:,} total injured, and {total_killed:,} total killed."

     return (
          fig_borough,
          fig_inj_borough,
          fig_factor,
          fig_pos,
          fig_year,
          fig_veh_pie,
          fig_person,
          fig_density,
          fig_map,
          fig_gender, # RE-ADDED
          fig_injury,
          fig_safety,
          fig_expanded,
          fig_emotional,
          fig_stacked,
          fig_time,
          fig_age_hist,
          fig_zip,
          fig_person_time,
          fig_ejection,
          summary,
     )


In [ ]:

ngrok.kill()
public_url = ngrok.connect(8050)
print("Dashboard running on:", public_url)

app.run_server(port=8050)


Dashboard running on: NgrokTunnel: "https://tentaculoid-daron-osteologically.ngrok-free.dev" -> "http://localhost:8050"


<IPython.core.display.Javascript object>